# Structural Analysis with pyMAOS
## Interactive Notebook for 2D Frame and Truss Analysis

This notebook provides an interactive interface for:
1. Loading structural models from YAML or JSON files
2. Visualizing the undeformed structure
3. Performing linear static analysis
4. Visualizing deformed shapes with results
5. Exporting results to Excel

## 1. Setup and Imports

In [5]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Check if we're in a Jupyter environment
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML
    JUPYTER_AVAILABLE = True
except ImportError:
    JUPYTER_AVAILABLE = False
    print("Note: ipywidgets not available. Install with: pip install ipywidgets")

# Add pyMAOS to path if needed
if os.path.exists('pyMAOS'):
    sys.path.insert(0, os.getcwd())

# Import pyMAOS modules
try:
    from pyMAOS.load_frame_from_file import load_frame_from_file
    from pyMAOS.structure2d import R2Structure
    from pyMAOS.loadcombos import LoadCombo
    from pyMAOS import unit_manager
    
    print("Imports successful")
except ImportError as e:
    print(f"Import error: {e}")
    print("Make sure you're running this from the pyMAOS_wUnits directory")
    raise

Imports successful


## 2. File Selection Widget

In [6]:
# Find all YAML and JSON files in current directory and subdirectories
structure_files = []

# Prioritize YAML files first
for root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith(('.yml', '.yaml', '.YML', '.YAML')):
            structure_files.append(os.path.join(root, file))

# Then add JSON files (excluding _SI suffix files)
for root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.json') and not file.endswith('_SI.json'):
            structure_files.append(os.path.join(root, file))

if not structure_files:
    print("No YAML or JSON files found in current directory")
    print("You can still enter a file path manually in the next cell")

if JUPYTER_AVAILABLE:
    # Create dropdown widget
    file_selector = widgets.Dropdown(
        options=structure_files if structure_files else ['No files found'],
        description='Structure File:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='700px')
    )

    # Or allow manual file path entry
    file_text = widgets.Text(
        placeholder='Or enter file path here...',
        description='File Path:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='700px')
    )

    load_button = widgets.Button(
        description='Load Structure',
        button_style='primary',
        icon='upload'
    )

    output = widgets.Output()

    display(HTML("<h3>Select Structure File (YAML preferred, JSON supported)</h3>"))
    display(file_selector)
    display(file_text)
    display(load_button)
    display(output)
else:
    print("Interactive widgets not available.")
    print("Available structure files (YAML and JSON):")
    for i, f in enumerate(structure_files[:15], 1):
        print(f"  {i}. {f}")
    if len(structure_files) > 15:
        print(f"  ... and {len(structure_files) - 15} more")

Dropdown(description='Structure File:', layout=Layout(width='700px'), options=('.\\.readthedocs.yaml', '.\\Bri…

Text(value='', description='File Path:', layout=Layout(width='700px'), placeholder='Or enter file path here...…

Button(button_style='primary', description='Load Structure', icon='upload', style=ButtonStyle())

Output()

## 3. Load Structure from File (YAML or JSON)

In [7]:
# Global variables to store structure
structure = None
loadcombos = None
current_file = None

def on_load_button_clicked(b):
    """Handler for load button click"""
    global structure, loadcombos, current_file
    
    if not JUPYTER_AVAILABLE:
        print("This function requires Jupyter widgets")
        return
    
    with output:
        output.clear_output()
        
        # Get file path from text input or dropdown
        file_path = file_text.value if file_text.value else file_selector.value
        
        if not file_path or file_path == 'No files found':
            print("Please select or enter a file path")
            return
        
        try:
            print(f"Loading structure from: {file_path}")
            
            # Use pyMAOS load_frame_from_file function
            nodes, members = load_frame_from_file(file_path)
            
            # Create structure from loaded nodes and members
            structure = R2Structure(nodes, members)
            
            # Create default load combination
            loadcombos = [LoadCombo("D", {"D": 1.0}, ["D"], False, "SLS")]
            
            current_file = file_path
            
            print(f"\nStructure loaded successfully!")
            print(f"  Nodes: {structure.NJ}")
            print(f"  Members: {structure.NM}")
            print(f"  DOF: {structure.NDOF}")
            print(f"\nContinue to the next cell to visualize")
            
        except Exception as e:
            print(f"Error loading structure: {e}")
            import traceback
            traceback.print_exc()

# Attach the handler if widgets are available
if JUPYTER_AVAILABLE:
    load_button.on_click(on_load_button_clicked)
    print("Load function ready")
else:
    print("To load a structure without widgets, use:")
    print("  nodes, members = load_frame_from_file('path/to/file.yml')")
    print("  structure = R2Structure(nodes, members)")
    print("  loadcombos = [LoadCombo('D', {'D': 1.0}, ['D'], False, 'SLS')]")
    print("  current_file = 'path/to/file.yml'")

Load function ready


## 4. Plot Undeformed Structure

In [8]:
if structure is None:
    print("Please load a structure first (run cells above)")
else:
    print("Plotting undeformed structure...")
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot members
    for member in structure.members:
        x = [member.inode.x, member.jnode.x]
        y = [member.inode.y, member.jnode.y]
        ax.plot(x, y, 'b-', linewidth=2, label='Member' if member.uid == 1 else '')
        
        mid_x = (member.inode.x + member.jnode.x) / 2
        mid_y = (member.inode.y + member.jnode.y) / 2
        ax.text(mid_x, mid_y, f'M{member.uid}', 
               bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7),
               ha='center', va='center')
    
    # Plot nodes
    for node in structure.nodes:
        if any(node.restraints):
            ax.plot(node.x, node.y, 'rs', markersize=10, label='Support' if node.uid == 1 else '')
        else:
            ax.plot(node.x, node.y, 'ko', markersize=8, label='Node' if node.uid == structure.nodes[1].uid else '')
        
        ax.text(node.x, node.y + 2, f'N{node.uid}', 
               ha='center', va='bottom', fontweight='bold')
        
        if node.restraints[0]:
            ax.plot([node.x-3, node.x], [node.y, node.y], 'r-', linewidth=3)
        if node.restraints[1]:
            ax.plot([node.x, node.x], [node.y-3, node.y], 'r-', linewidth=3)
    
    # Plot loads
    for node in structure.nodes:
        for load_case, load in node.loads.items():
            fx, fy, mz = load
            if hasattr(fx, 'magnitude'):
                fx = fx.magnitude
                fy = fy.magnitude
            if abs(fx) > 1e-6 or abs(fy) > 1e-6:
                ax.arrow(node.x, node.y, fx*0.5, fy*0.5, 
                        head_width=2, head_length=1, fc='green', ec='green')
    
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Undeformed Structure', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal', 'box')
    ax.legend()
    plt.tight_layout()
    plt.show()
    
    print("Plot complete")

Please load a structure first (run cells above)


## 5. Perform Analysis

In [ ]:
if structure is None:
    print("Please load a structure first")
else:
    print("Performing linear static analysis...\n")
    
    try:
        for combo in loadcombos:
            print(f"Analyzing load combination: {combo.name}")
            
            U = structure.solve_linear_static(combo, verbose=False)
            structure.set_node_displacements(combo)
            structure.compute_reactions(combo)
            
            for member in structure.members:
                member.set_end_forces_global(combo)
                if member.type != "TRUSS":
                    member.Flocal(combo)
            
            print(f"  Analysis complete for {combo.name}\n")
        
        print("\n" + "="*60)
        print("ANALYSIS RESULTS SUMMARY")
        print("="*60)
        
        print("\nNode Displacements:")
        print("-" * 60)
        for node in structure.nodes:
            if combo.name in node.displacements:
                disp = node.displacements[combo.name]
                print(f"Node {node.uid}: Ux={disp[0]:.6e}  Uy={disp[1]:.6e}  Rz={disp[2]:.6e}")
        
        print("\nReactions:")
        print("-" * 60)
        for node in structure.nodes:
            if any(node.restraints) and combo.name in node.reactions:
                react = node.reactions[combo.name]
                print(f"Node {node.uid}: Rx={react[0]:.6e}  Ry={react[1]:.6e}  Mz={react[2]:.6e}")
        
        print("\n" + "="*60)
        print("All analyses complete!")
        print("Continue to next cell to plot deformed shape")
        
    except Exception as e:
        print(f"\nError during analysis: {e}")
        import traceback
        traceback.print_exc()

## 6. Plot Deformed Structure

In [ ]:
if structure is None:
    print("Please load a structure first")
elif not hasattr(structure, 'U') or structure.U is None:
    print("Please run the analysis first (cell above)")
elif not JUPYTER_AVAILABLE:
    print("Interactive plotting requires ipywidgets")
    print("Plotting with default scale of 100x...")
    scale = 100
    
    fig, ax = plt.subplots(figsize=(12, 8))
    combo = loadcombos[0]
    
    for member in structure.members:
        x = [member.inode.x, member.jnode.x]
        y = [member.inode.y, member.jnode.y]
        ax.plot(x, y, 'gray', linewidth=1, linestyle='--', alpha=0.5, 
               label='Undeformed' if member.uid == 1 else '')
    
    for member in structure.members:
        i_disp = member.inode.displacements.get(combo.name, [0, 0, 0])
        j_disp = member.jnode.displacements.get(combo.name, [0, 0, 0])
        
        if hasattr(i_disp[0], 'magnitude'):
            i_ux, i_uy = i_disp[0].magnitude, i_disp[1].magnitude
            j_ux, j_uy = j_disp[0].magnitude, j_disp[1].magnitude
        else:
            i_ux, i_uy = i_disp[0], i_disp[1]
            j_ux, j_uy = j_disp[0], j_disp[1]
        
        x_def = [member.inode.x + scale*i_ux, member.jnode.x + scale*j_ux]
        y_def = [member.inode.y + scale*i_uy, member.jnode.y + scale*j_uy]
        ax.plot(x_def, y_def, 'b-', linewidth=2.5, 
               label='Deformed' if member.uid == 1 else '')
    
    for node in structure.nodes:
        disp = node.displacements.get(combo.name, [0, 0, 0])
        if hasattr(disp[0], 'magnitude'):
            ux, uy = disp[0].magnitude, disp[1].magnitude
        else:
            ux, uy = disp[0], disp[1]
        
        x_def = node.x + scale*ux
        y_def = node.y + scale*uy
        
        if any(node.restraints):
            ax.plot(x_def, y_def, 'rs', markersize=10)
        else:
            ax.plot(x_def, y_def, 'ko', markersize=8)
        
        ax.text(x_def, y_def + 2, f'N{node.uid}', 
               ha='center', va='bottom', fontweight='bold')
    
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title(f'Deformed Structure (Scale: {scale}x)', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal', 'box')
    ax.legend()
    plt.tight_layout()
    plt.show()
else:
    scale_slider = widgets.FloatSlider(
        value=100,
        min=1,
        max=1000,
        step=10,
        description='Displacement Scale:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    
    def plot_deformed(scale):
        fig, ax = plt.subplots(figsize=(12, 8))
        combo = loadcombos[0]
        
        for member in structure.members:
            x = [member.inode.x, member.jnode.x]
            y = [member.inode.y, member.jnode.y]
            ax.plot(x, y, 'gray', linewidth=1, linestyle='--', alpha=0.5, 
                   label='Undeformed' if member.uid == 1 else '')
        
        for member in structure.members:
            i_disp = member.inode.displacements.get(combo.name, [0, 0, 0])
            j_disp = member.jnode.displacements.get(combo.name, [0, 0, 0])
            
            if hasattr(i_disp[0], 'magnitude'):
                i_ux = i_disp[0].magnitude
                i_uy = i_disp[1].magnitude
                j_ux = j_disp[0].magnitude
                j_uy = j_disp[1].magnitude
            else:
                i_ux, i_uy = i_disp[0], i_disp[1]
                j_ux, j_uy = j_disp[0], j_disp[1]
            
            x_def = [member.inode.x + scale*i_ux, member.jnode.x + scale*j_ux]
            y_def = [member.inode.y + scale*i_uy, member.jnode.y + scale*j_uy]
            ax.plot(x_def, y_def, 'b-', linewidth=2.5, 
                   label='Deformed' if member.uid == 1 else '')
        
        for node in structure.nodes:
            disp = node.displacements.get(combo.name, [0, 0, 0])
            
            if hasattr(disp[0], 'magnitude'):
                ux = disp[0].magnitude
                uy = disp[1].magnitude
            else:
                ux, uy = disp[0], disp[1]
            
            x_def = node.x + scale*ux
            y_def = node.y + scale*uy
            
            if any(node.restraints):
                ax.plot(x_def, y_def, 'rs', markersize=10)
            else:
                ax.plot(x_def, y_def, 'ko', markersize=8)
            
            ax.text(x_def, y_def + 2, f'N{node.uid}', 
                   ha='center', va='bottom', fontweight='bold')
        
        ax.set_xlabel('X Coordinate')
        ax.set_ylabel('Y Coordinate')
        ax.set_title(f'Deformed Structure (Scale: {scale}x)', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal', 'box')
        ax.legend()
        plt.tight_layout()
        plt.show()
    
    interactive_plot = widgets.interactive(plot_deformed, scale=scale_slider)
    display(interactive_plot)

## 7. Export Results to Excel

In [ ]:
if structure is None:
    print("Please load a structure first")
elif not hasattr(structure, 'U') or structure.U is None:
    print("Please run the analysis first")
else:
    output_path = Path(current_file).parent / (Path(current_file).stem + '_results.xlsx')
    
    print(f"Exporting results to: {output_path}\n")
    
    try:
        result = structure.export_results_to_excel(
            output_file=str(output_path),
            loadcombos=loadcombos,
            include_visualization=True
        )
        
        print(f"Results exported successfully!")
        print(f"  File: {result}")
        
    except Exception as e:
        print(f"Error exporting results: {e}")
        import traceback
        traceback.print_exc()

## 8. Summary Report

In [ ]:
if structure is not None:
    summary = structure.get_summary()
    print(summary)
else:
    print("Please load a structure first")